In [1]:
import pandas as pd

In [5]:
bad_sent = pd.read_csv('/Users/venkatasaisumanthsadu/Documents/clg-documents/CSCI544/project/CSCI544_NLP_Project-main/bad_sentences.csv')
good_sent = pd.read_csv('/Users/venkatasaisumanthsadu/Documents/clg-documents/CSCI544/project/CSCI544_NLP_Project-main/good_sentences.csv')

In [10]:
bad_sent = bad_sent[['comments','labels']]
good_sent = good_sent[['comments','labels']]

In [11]:
bad_sent = bad_sent.sample(10)
bad_sent

,comments,labels
5769,And none of this white-collar resort prison. T...,1
4829,This article from last month still mentions th...,1
784,an inconsequential person that conspiracy theo...,1
6733,"Canons, repeating rifles, warships, and even m...",1
9216,"As somebody who has done basically that, it's ...",1
12016,There will never be peace with leaders like Bi...,1
12191,Get the fuck out of here they were dickriding ...,1
2502,You stan hard enough for Bread Cruz to know he...,1
3261,"To be clear, I think he is a scum bad, sex tra...",1
3277,"But one of the girls did name his ""co-conspira...",1


In [9]:
good_sent_vectors = []
bad_sent_vectors = []

,comments,labels
0,I get this feeling that derailments happen A L...,1
1,1) pay your workers less than your shareholder...,1
2,> Former US president Barack Obama passed legi...,1
3,Guess the US government is in the process of f...,1
4,Nationalize the rail. This is happening becaus...,1
...,...,...
14434,"eh, sucks for everyone yes, i agree, but it su...",1
14435,Russia does have a larger and higher tech air ...,1
14436,If there's any finalists in the olympics of su...,1
14437,"Mostly through reddit and the sources linked, ...",1


In [27]:
bad_sent[['comments']]

,comments
5769,And none of this white-collar resort prison. T...
4829,This article from last month still mentions th...
784,an inconsequential person that conspiracy theo...
6733,"Canons, repeating rifles, warships, and even m..."
9216,"As somebody who has done basically that, it's ..."
12016,There will never be peace with leaders like Bi...
12191,Get the fuck out of here they were dickriding ...
2502,You stan hard enough for Bread Cruz to know he...
3261,"To be clear, I think he is a scum bad, sex tra..."
3277,"But one of the girls did name his ""co-conspira..."


In [ ]:
# Program to measure the similarity between
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# X = input("Enter first string: ").lower()
# Y = input("Enter second string: ").lower()
# X ="I love horror movies"
# Y ="Lights out is a horror movie"

# tokenization
# for i in range(len(bad_sent)):
# 	X_list = word_tokenize(bad_sent)
# 	Y_list = word_tokenize(Y)

for row in bad_sent.itertuples(index = True):
	X_list = word_tokenize(row[1])
	bad_sent_vectors.append(X_list)

for good_rows in good_sent.itertuples(index = True):
	Y_list = word_tokenize(good_rows[1])
	good_sent_vectors.append(Y_list)



# # sw contains the list of stopwords
# sw = stopwords.words('english')
# l1 =[];l2 =[]

# # remove stop words from the string
# X_set = {w for w in X_list if not w in sw}
# Y_set = {w for w in Y_list if not w in sw}

# form a set containing keywords of both strings
rvector = X_set.union(Y_set)
for w in rvector:
	if w in X_set: l1.append(1) # create a vector
	else: l1.append(0)
	if w in Y_set: l2.append(1)
	else: l2.append(0)
c = 0

# cosine formula
for i in range(len(rvector)):
		c+= l1[i]*l2[i]
cosine = c / float((sum(l1)*sum(l2))**0.5)
print("similarity: ", cosine)


In [31]:
bad_comments = []
good_comments = []
for bad_rows in bad_sent.itertuples(index = True):
    bad_comments.append(bad_rows[1])

for good_rows in good_sent.itertuples(index = True):
    good_comments.append(good_rows[1])


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define two sample documents
# doc1 = "The quick brown fox jumps over the lazy dog."
# doc2 = "A quick brown dog outpaces a quick fox."

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create a matrix of tf-idf scores for the two documents
tfidf_matrix_bad = tfidf_vectorizer.fit_transform(bad_comments)
tfidf_matrix_good = tfidf_vectorizer.fit_transform(good_comments)

# Calculate the cosine similarity between the two documents
cosine_sim = cosine_similarity(tfidf_matrix_bad[0], tfidf_matrix_good[0])[0][0]

print("Cosine similarity:", cosine_sim)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 446 while Y.shape[1] == 32281

In [36]:
tfidf_matrix_bad.shape

(10, 446)

In [40]:
tfidf_matrix_good.shape

(37264, 32281)

In [43]:
import gensim.downloader as api
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

In [44]:
wv = api.load('word2vec-google-news-300')

In [45]:
import numpy as np

In [46]:
# converts sentences/reivews to array of list of words for them to be converted into word2vec vectors later.
def sentence_to_words(sentences):
    result_list = []
    for sent in sentences:
        result_list.append(utils.simple_preprocess(sent))
    
    return np.asarray(result_list, dtype=object)

# find the average vector for each sentence/review by adding all the vectors of words in a sentence/review and dividing by lenght of the sentence (no of words in sentence/review)
def avg_vector(sent, wv):
    vector = np.zeros((300,))
    if len(sent) != 0:
        for word in sent:
            if word in wv:
                vector = vector + wv[word]

        vector = vector / len(sent)
    
    return vector
    
# find average vector for each sentence/review in corpus.
def review_to_vector(sentences, wv):
    result = np.empty((len(sentences), 300), dtype=np.float64)
    for i, sent in enumerate(sentences):
        result[i] = avg_vector(sent, wv)
        
    return result

In [47]:
x_bad_wv = sentence_to_words(bad_comments)
x_good_wv = sentence_to_words(good_comments)

# final vectors for each review
x_bad_wv = review_to_vector(x_bad_wv, wv)
x_good_wv = review_to_vector(x_good_wv, wv)

In [50]:
x_bad_wv[0]

array([ 7.32421875e-05, -5.79566956e-03,  6.52282715e-02,  7.81463623e-02,
        3.94836426e-02, -4.98291016e-02, -1.13739014e-02, -1.80725098e-02,
        3.69506836e-02,  4.86939430e-02, -6.63986206e-03, -7.82775879e-02,
       -2.25708008e-02,  1.13342285e-02, -1.18554688e-01,  4.36798096e-02,
       -3.08227539e-04,  8.76579285e-02, -3.88671875e-02, -4.81933594e-02,
        1.26296997e-02, -6.93054199e-03,  1.45034790e-03, -2.14038849e-02,
        4.86846924e-02, -7.27401733e-02, -7.35023499e-02,  5.01403809e-02,
        5.59852600e-02, -1.10107422e-02,  1.79687500e-02,  1.65618896e-02,
       -3.38928223e-02,  4.13831711e-02, -2.98271179e-02,  1.99779510e-02,
        4.60059166e-02, -2.73864746e-02, -3.16070557e-02,  8.13796997e-02,
        5.64331055e-02, -7.00546265e-02,  9.27734375e-02, -3.70697021e-02,
       -1.97784424e-02,  1.51748657e-03,  1.61514282e-02,  1.33117676e-02,
       -1.86737061e-02,  3.19213867e-02, -4.06494141e-03,  4.25140381e-02,
        2.34588623e-02, -

In [51]:
x_good_wv[0]

array([ 8.20289025e-02,  1.14652194e-02,  5.94764123e-02,  1.12267127e-01,
       -2.35877404e-02,  3.07203440e-02,  1.44418570e-02, -1.38038048e-01,
        3.64473783e-02,  1.40596243e-01, -9.13837139e-02, -1.62250225e-01,
       -2.34175462e-02,  1.63762019e-02, -2.16533954e-02,  6.52547983e-02,
       -1.67001578e-02,  9.54402043e-02,  3.67431641e-02, -4.39265325e-02,
       -1.90241887e-02,  2.05136813e-02,  7.12421124e-02, -2.28107159e-02,
       -2.36440805e-02,  8.81723257e-03, -1.01975661e-01,  8.71722882e-02,
        3.81446252e-02,  7.61718750e-02,  2.53389799e-02,  5.92839168e-02,
        4.80393630e-02, -3.06493319e-02,  4.03864934e-02,  9.37822782e-02,
       -5.32367413e-02,  5.92510517e-03,  1.29077618e-01,  1.08022837e-01,
        2.12026743e-02, -6.40373230e-02,  4.33819111e-02,  5.17859826e-03,
       -2.51277043e-02, -2.43665255e-02,  5.23399940e-02,  1.89208984e-03,
       -1.28220778e-02,  4.62380923e-02, -3.13415527e-02,  9.70599835e-02,
       -3.03790753e-02,  

In [58]:
cosine_similarity([x_bad_wv[0]], [x_good_wv[0]])[0][0]

0.5261903431382597

In [ ]:
max_similar = 0
indx = 0
 
sent_list = []

for i in range(len(good_comments)):
    sent_list.append((i, cosine_similarity([x_bad_wv[0]], [x_good_wv[i]])[0][0]))
    # if cosine_similarity([x_bad_wv[0]], [x_good_wv[i]])[0][0] > max_similar:
    #     max_similar = cosine_similarity([x_bad_wv[0]], [x_good_wv[i]])[0][0]
    #     indx = i

# print(indx, max_similar)
sent_list

In [69]:

# Python program to sort a list of tuples by the second Item
  
# Function to sort the list of tuples by its second item
  
  
def Sort_Tuple(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    tup.sort(key = lambda x: x[1]) 
    return tup 
  
  
# Driver Code
  
sorted_list = Sort_Tuple(sent_list)

In [70]:
sorted_list[-10:]

[(19275, 0.793549910664419),
 (19969, 0.7960331177452454),
 (18716, 0.7965489076563084),
 (9730, 0.8011442266448724),
 (9984, 0.8020099049361524),
 (16508, 0.8038161591069598),
 (9690, 0.8067328679938854),
 (9931, 0.8120637785688658),
 (20497, 0.8130163622612663),
 (15091, 0.8240306686920722)]

In [65]:
bad_comments[0]

'And none of this white-collar resort prison. They should go straight to federal pound them in the ass prison.'

In [64]:
good_comments[15091]

'And not white collar golfing prison either. They need to be in general population.'